In [1]:
import os

import itertools
import pickle
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import math 

from torchtext import data, datasets
from pathlib import Path

import sys
sys.path.append('../')
%matplotlib inline

In [2]:
# Init cuda
device = "cuda:6" if torch.cuda.is_available() else "cpu"
idevice = 6 if torch.cuda.is_available() else -1
torch.cuda.set_device(idevice)
device, idevice

('cuda:6', 6)

In [3]:
TEXT = data.Field()
PATH = Path('./wikitext/')

In [4]:
batch_size = 256
sequence_length = 10
grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 10_000

In [5]:
from torchtext.datasets.language_modeling import LanguageModelingDataset

class WikiTextRu(LanguageModelingDataset):

    urls = ['http://files.deeppavlov.ai/datasets/wikitext_ru.zip']
    name = 'wikitext_ru'
    dirname = 'wikitext_ru'

    @classmethod
    def splits(cls, text_field, root='.data', 
               train='ru.wiki.train.txt', validation='ru.wiki.valid.txt', test='ru.wiki.test.txt',  **kwargs):

        return super().splits(text_field=text_field, root=root, 
                              train=train, validation=validation, test=test, **kwargs)

    @classmethod
    def iters(cls, batch_size=32, bptt_len=35, device=0, root='.data', vectors=None, **kwargs):
       
        TEXT = data.Field()
        train, val, test = cls.splits(TEXT, root=root, **kwargs)
        TEXT.build_vocab(train, vectors=vectors)

        return data.BPTTIterator.splits((train, val, test), 
                                        batch_size=batch_size, bptt_len=bptt_len, device=device)

In [6]:
from torchtext.vocab import Vectors

class RuFastText(Vectors):

    url_base = 'http://files.deeppavlov.ai/embeddings/ft_native_300_ru_wiki_lenta_nltk_word_tokenize/ft_native_300_ru_wiki_lenta_nltk_word_tokenize.vec'

    def __init__(self, **kwargs):
        url = self.url_base
        name = os.path.basename(url)
        super().__init__(name, url=url, **kwargs)


In [7]:
%%time
train_ds, valid_ds, test_ds = WikiTextRu.splits(TEXT, root=PATH)

CPU times: user 1min 25s, sys: 23.8 s, total: 1min 49s
Wall time: 1min 49s


In [8]:
ru_vectors = RuFastText()

In [9]:
%%time
TEXT.build_vocab(train_ds, min_freq=30, max_size=20_000, vectors=ru_vectors)

CPU times: user 2min 53s, sys: 19.2 s, total: 3min 12s
Wall time: 3min 11s


In [10]:
ntokens = len(TEXT.vocab)
ntokens

20002

In [11]:
%%time
train_loader, val_loader, test_loader = data.BPTTIterator.splits((train_ds, valid_ds, test_ds), 
                                        batch_sizes=(batch_size, batch_size, batch_size), 
                                        bptt_len=sequence_length, repeat=False, device=idevice) 

CPU times: user 153 µs, sys: 30 µs, total: 183 µs
Wall time: 190 µs


In [12]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, emb_vectors, nhid, nlayers, dropout=0.5):
        super().__init__()
        ntoken = emb_vectors.shape[0]
        ninp = emb_vectors.shape[1]
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.encoder.weight.data.copy_(emb_vectors);
        self.encoder.weight.requires_grad = False
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [13]:
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    ntokens = len(TEXT.vocab)
    hidden = model.init_hidden(batch_size)
    for i, b in enumerate(data_loader):
        data, targets = b.text, b.target
        output, hidden = model(data)
        output_flat = output.view(-1, ntokens)
        total_loss += criterion(output_flat, targets.view(-1)).item()
    return total_loss / len(data_loader)

In [14]:
def train():
    model.train()
    total_loss = 0
    ntokens = len(TEXT.vocab) #len(corpus.dictionary)
    for batch, b in enumerate(train_loader):
        data, targets = b.text, b.target
        model.zero_grad()
        output, hidden = model(data)
        loss = criterion(output.view(-1, ntokens), targets.view(-1))
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), grad_clip)
        for p in filter(lambda p: p.requires_grad, model.parameters()):
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_loader), lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [15]:
def generate(n=50, temp=1.):
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long().to(device)
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
        s = TEXT.vocab.itos[s_idx]
        out.append(s)
    return ' '.join(out)

In [16]:
emb_vectors = TEXT.vocab.vectors
model = RNNModel('GRU', emb_vectors, 1024, 1, 0.0).to(device)
criterion = nn.CrossEntropyLoss()

In [17]:
%%time
with torch.no_grad():
    print('sample:\n', generate(15), '\n')

sample:
 направился мимо моей орденов журналы степной за началось песне документ дорог Португалии. погиб Екатерина естественным 

CPU times: user 13.2 ms, sys: 4.24 ms, total: 17.4 ms
Wall time: 16.6 ms


In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
%%time
for epoch in range(1, 2):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')


| epoch   1 | 10000/149747 batches | lr 4.00 | loss  5.58 | ppl   265.13
| epoch   1 | 20000/149747 batches | lr 4.00 | loss  5.10 | ppl   164.06
| epoch   1 | 30000/149747 batches | lr 4.00 | loss  4.90 | ppl   133.73
| epoch   1 | 40000/149747 batches | lr 4.00 | loss  4.77 | ppl   117.46
| epoch   1 | 50000/149747 batches | lr 4.00 | loss  4.68 | ppl   108.16
| epoch   1 | 60000/149747 batches | lr 4.00 | loss  4.61 | ppl   100.02
| epoch   1 | 70000/149747 batches | lr 4.00 | loss  4.56 | ppl    95.68
| epoch   1 | 80000/149747 batches | lr 4.00 | loss  4.51 | ppl    91.12
| epoch   1 | 90000/149747 batches | lr 4.00 | loss  4.46 | ppl    86.83
| epoch   1 | 100000/149747 batches | lr 4.00 | loss  4.43 | ppl    83.93
| epoch   1 | 110000/149747 batches | lr 4.00 | loss  4.40 | ppl    81.85
| epoch   1 | 120000/149747 batches | lr 4.00 | loss  4.38 | ppl    79.81
| epoch   1 | 130000/149747 batches | lr 4.00 | loss  4.36 | ppl    78.14
| epoch   1 | 140000/149747 batches | lr 4.00 |

In [20]:
%%time
for epoch in range(2, 6):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')



| epoch   2 | 10000/149747 batches | lr 4.00 | loss  4.29 | ppl    72.91
| epoch   2 | 20000/149747 batches | lr 4.00 | loss  4.27 | ppl    71.49
| epoch   2 | 30000/149747 batches | lr 4.00 | loss  4.25 | ppl    70.18
| epoch   2 | 40000/149747 batches | lr 4.00 | loss  4.24 | ppl    69.07
| epoch   2 | 50000/149747 batches | lr 4.00 | loss  4.23 | ppl    68.51
| epoch   2 | 60000/149747 batches | lr 4.00 | loss  4.20 | ppl    66.88
| epoch   2 | 70000/149747 batches | lr 4.00 | loss  4.20 | ppl    66.93
| epoch   2 | 80000/149747 batches | lr 4.00 | loss  4.19 | ppl    66.17
| epoch   2 | 90000/149747 batches | lr 4.00 | loss  4.17 | ppl    64.95
| epoch   2 | 100000/149747 batches | lr 4.00 | loss  4.16 | ppl    64.31
| epoch   2 | 110000/149747 batches | lr 4.00 | loss  4.16 | ppl    64.10
| epoch   2 | 120000/149747 batches | lr 4.00 | loss  4.15 | ppl    63.65
| epoch   2 | 130000/149747 batches | lr 4.00 | loss  4.15 | ppl    63.37
| epoch   2 | 140000/149747 batches | lr 4.00 |